In [5]:
from ortools.sat.python import cp_model
import numpy as np
import networkx as nx
import itertools as it
import random

Data generation

In [6]:
def distance_2d(point_1, point_2):
    return ((point_1[0] - point_2[0])**2 + (point_1[1] - point_2[1])**2)**(1/2)


def generate_data_tsp(num_of_nodes, area=(0, 10000), is_complete=True):
    distr_by_x = np.random.uniform(*area, num_of_nodes)
    distr_by_y = np.random.uniform(*area, num_of_nodes)
    points = [(i, j) for i, j in zip(distr_by_x, distr_by_y)]

    distance_matrix = [[distance_2d(points[i], points[j]) for j in range(0, num_of_nodes)] for i in range(0, num_of_nodes)]

    if not is_complete:
        for i in range(0, num_of_nodes):
            to_remove = list(np.random.randint(0, num_of_nodes - 1, num_of_nodes//random.randint(8, 10)))
            while to_remove:
                curr = int(random.randint(0, num_of_nodes - 1))
                if curr != i:
                    distance_matrix[i][curr] = area[1] * 10000
                    to_remove.pop(0)
    return distance_matrix

Greedy and exact solvers

In [8]:
def get_sequence(matrix):
    iterations = 0
    index = 0
    sequence = [index]
    while iterations != len(matrix):
        for j in range(0, len(matrix)):
            if matrix[index][j] == 1:
                sequence.append(j)
                index = j
                break
        iterations += 1
    return sequence


def solve_tsp_problem(adj_matrix, method='exact'):
    if method == 'greedy':
        print('Greedy algorithm solution.')
        check = [[i, 0] for i in range(0, len(adj_matrix))]

        start_vertex = random.randint(0, len(adj_matrix) - 1)
        curr = start_vertex
        check[start_vertex][1] = 1
        num_of_unchecked = len(check) - 1
        objective_value = 0
        seq = [start_vertex]
        while num_of_unchecked > 0:
            minn = float('inf')
            index = -1
            for j in range(0, len(adj_matrix)):
                if check[j][1] == 0 and adj_matrix[curr][j] < minn:
                    minn = adj_matrix[curr][j]
                    index = j
            objective_value += minn
            seq.append(index)
            check[index][1] = 1
            num_of_unchecked -= 1
            curr = index
        seq.append(start_vertex)
        objective_value += adj_matrix[curr][start_vertex]

        print(f'Target function: {objective_value}')
        print(f'Sequence: {"->".join(map(str, seq))}')
        print('--------------------')
        return objective_value, seq

    else:
        print('Exact algorithm solution.')
        m = cp_model.CpModel()

        # all vars
        variables = []
        for i in range(0, len(adj_matrix)):
            v = []
            for j in range(0, len(adj_matrix)):
                x = m.new_bool_var(f'x_{i},{j}')
                v.append(x)
            variables.append(v)

        # diagonal elements are zeros
        for i in range(0, len(adj_matrix)):
            m.add(variables[i][i] == 0)

        # one 1 in each row
        for i in range(0, len(adj_matrix)):
            constraints = sum([variables[i][j] for j in range(0, len(adj_matrix))])
            #print(constraints)
            m.add(constraints == 1)

        # one 1 in each column
        for j in range(0, len(adj_matrix)):
            constraints = sum([variables[i][j] for i in range(0, len(adj_matrix))])
            #print(constraints)
            m.add(constraints == 1)

        # function to minimize
        opt_func = sum(variables[i][j] * adj_matrix[i][j] for i in range(0, len(adj_matrix)) for j in range(0, len(adj_matrix)))

        m.minimize(opt_func)
        solver = cp_model.CpSolver()
        status = solver.solve(m)

        # current matrix
        current = []
        for i in range(0, len(adj_matrix)):
            cur = []
            for j in range(0, len(adj_matrix)):
                cur.append(solver.value(variables[i][j]))
            current.append(cur)

        iterations = 0

        # while num of connected components > 1: add constraints
        while len(list(nx.connected_components(nx.from_numpy_array(np.array(current))))) > 1 and iterations < 100:
            subtours = list(nx.connected_components(nx.from_numpy_array(np.array(current))))
            iterations += 1
            minn = len(adj_matrix) + 1
            index = -1
            for i in range(0, len(subtours)):
                if len(subtours[i]) < minn:
                    minn = len(subtours[i])
                    index = i
            combinations = list(it.combinations(subtours[index], 2))
            constraint = sum([variables[i][j] + variables[j][i] for i, j in combinations])
            m.add(constraint <= minn - 1)

            status = solver.solve(m)

            current = []
            for i in range(0, len(adj_matrix)):
                cur = []
                for j in range(0, len(adj_matrix)):
                    cur.append(solver.value(variables[i][j]))
                current.append(cur)

        for cur in current:
            print(cur)

        seq = get_sequence(current)
        print(f'Iterations: {iterations}')
        print(f'Target function: {solver.objective_value}')
        print(f'Sequence: {"->".join(map(str, seq))}')
        print('--------------------')
        return solver.objective_value, seq


if __name__ == '__main__':
    a = generate_data_tsp(10)
    for i in a:
        print(i)
    solve_tsp_problem(a, 'greedy')
    solve_tsp_problem(a)


[np.float64(0.0), np.float64(4563.773360797299), np.float64(4306.064334544671), np.float64(3142.5375563461776), np.float64(6100.205264466974), np.float64(2438.6792706592064), np.float64(7128.63481299334), np.float64(2317.0545331444837), np.float64(5819.814598389157), np.float64(2489.3492244120157)]
[np.float64(4563.773360797299), np.float64(0.0), np.float64(8289.38977980162), np.float64(4544.888606856384), np.float64(8836.50823101008), np.float64(6708.425343108844), np.float64(10543.23673677619), np.float64(3039.5263913613726), np.float64(9868.788008069867), np.float64(6095.306273789904)]
[np.float64(4306.064334544671), np.float64(8289.38977980162), np.float64(0.0), np.float64(4316.3697409545775), np.float64(3004.5490891911104), np.float64(4434.938610090705), np.float64(3012.461264113136), np.float64(6617.3607757073305), np.float64(1580.2210114090758), np.float64(2199.3468336841192)]
[np.float64(3142.5375563461776), np.float64(4544.888606856384), np.float64(4316.3697409545775), np.floa